### Project 3

### Classification Modeling on Subreddits: Futurists / Scientists

#### Libary imports

In [72]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


import warnings; warnings.simplefilter('ignore')
##this will hide deprecation/future warnings

from IPython.display import Markdown, display
pd.set_option('display.max_row', 200) # Set ipython's max row display
pd.set_option('display.max_columns', 85) # Set iPython's max column count
pd.set_option('display.max_colwidth', 1_000) # Set iPython's max column width

# pseudo-markdown in code cells
def printmd(string):
    display(Markdown(string))
# ref: https://discuss.analyticsvidhya.com/t/how-to-make-a-text-bold-within-print-statement-in-ipython-notebook/14552/2

#### Data collection

I utilized the Python Reddit API Wrapper (PRAW) API in the data collection process. You can find the steps I took for that in the `PRAW_data_collection` notebook, located in the code folder of this repository.

The East Coast local instructors were very generous with their walkthrough of the process, so I definitely credit them for the ease of the data collection process.

#### Preprocessing and EDA

In [31]:
subred1 = pd.read_csv('../data/df_with_both_subs.csv', index_col = 'id')
subred1 = subred1.drop(columns = 'Unnamed: 0')
display(pd.set_option('display.max_colwidth', 50), subred1.head())

None

,title,score,url,comms_num,created,body,subreddit
id,,,,,,,
japxs3,What would the point be to do anything if AI c...,0,https://www.reddit.com/r/Futurology/comments/j...,4,1.602664e+09,The more I look into AI and new projects like ...,futurology
japq75,ELCC Explained: the Critical Renewable Energy ...,4,https://blog.ucsusa.org/mark-specht/elcc-expla...,0,1.602663e+09,NaN,futurology
jaojnb,There's a 50-50 chance we're living in a simul...,8,https://boingboing.net/2020/10/13/new-research...,4,1.602659e+09,NaN,futurology
jaofpy,"Eight nations sign NASA's Artemis Accords, ple...",3,https://www.engadget.com/nasa-artemis-accords-...,1,1.602658e+09,NaN,futurology
jaocqq,Mercedes benz AVTR - In Action,8,https://www.youtube.com/watch?v=ChqM3zqTREQ&ab...,0,1.602658e+09,NaN,futurology


---

In [24]:
printmd('**Value counts:**')
display(subred1['subreddit'].value_counts())

printmd('**Value counts by weight:**')
subred1['subreddit'].value_counts(normalize = True)

**Value counts:**

science       931
futurology    869
Name: subreddit, dtype: int64

**Value counts by weight:**

science       0.517222
futurology    0.482778
Name: subreddit, dtype: float64

* I may want to return to the subreddits to get a bigger dataset to work with. 

* The classes are slightly unbalanced, so that will be a consideration during the preprocessing / get-more-data phase.

In [33]:
subred1[['title', 'subreddit']]

,title,subreddit
id,,
japxs3,What would the point be to do anything if AI c...,futurology
japq75,ELCC Explained: the Critical Renewable Energy ...,futurology
jaojnb,There's a 50-50 chance we're living in a simul...,futurology
jaofpy,"Eight nations sign NASA's Artemis Accords, ple...",futurology
jaocqq,Mercedes benz AVTR - In Action,futurology
...,...,...
ir136p,Adults with positive SARS-CoV-2 test results w...,science
ir0vem,50% of Phosphorus Lost to Erosion,science
ir0pvm,Motivated Helplessness in the Context of the C...,science


In [74]:
# Null body rows
printmd(f"**Only {len(subred1['body'][(subred1['body'].isnull()).astype(int) == 0])} non-null `body` records. Can we get enough content from post titles?**")

non_null_body = subred1[['body', 'subreddit']][(subred1['body'].isnull()).astype(int) == 0]
display(non_null_body)

**Only 69 non-null `body` records. Can we get enough content from post titles?**

,body,subreddit
id,,
japxs3,"The more I look into AI and new projects like GPT-3, we humans suck. Sure GPT-3 isn't on the level as a human brain, but that doesn't mean gpt-4 or gpt-5 won't be. I can already see the massive jobs loss as AI will offset more jobs than it will create. I'm just feeling a little existential crisis here",futurology
jamgnw,"As our dependence on computer technologies increases, will kids who were not allowed to use electronics (phones, computers, tablets) as toddlers become stunted because they wouldn't know how to use computers?\n\nWill computers be so ingrained in society that toddlers need to learn how to use them just as how they need to learn language?\n\nBy this metric, is it child abuse if you DON'T teach your kids how to use computers?",futurology
jag7ht,Please post all climate change news here unless the submission is an unique event that is a global headline across several trusted news sources.,futurology
ja7ygc,"Is there any effort by others to bring the other technologies such as, Molecular Manufacturing into reality?",futurology
ja5exx,"Everyday we're learing more efficient ways to synthesize food. Eventually there's going to be a synthetic dietary movement similar to vegetarianism and veganism. Probably sooner rather than later. And it won't be restricted to meat, synthetic vegetables and fruit will be included.\n\nSo what would a term for people who only eat synthetic food be? Artificivore? Synthetivore? Synthetarian? \n\nPitch ideas bellow.",futurology
j9wqlc,"First off I consider this to be more of an experiment in terms of this type of AI. I have no idea if this prediction is valid, but I also think it would be interesting to try and fact check this. I was wondering if we could systematically test this GPT3 AI, and effectively test our collective scientific understanding. Remember this thing learned based on posts all over Reddit including Futurology. So if you have posted then a part of you is in this thing. Would there be a way to systematically probe GPT3s general scientific understanding?\n\n\n\n\nPrompt --->\nHow dense is planet 9?\n\n\nAnswer --->\nThe planet called 9 is actually a celestial body that orbits the sun. Like all celestial bodies, it has its own gravitational pull which causes things to fall towards it and hence creates density. Therefore, if you were to put an object on this planet's surface then it would become more dense depending upon how close or far away from the center of gravity they are.\n\nThe center of gra...",futurology
j9tzw6,"I think if we had high skills, we might not need artificial intelligence so much. For example:\n\nSuppose you can upload different languages of the world with the highest accuracy to your brain in a few minutes, so what is the need for artificial intelligence to do translation? Or that anyone can upload all kinds of science to their brain.\n\nMaybe we will eventually need machines that help us because of our physical weakness, such as self-driving cars. We need self-driving because we can't drive for 48 hours straight. This is physical weakness. But with a BCI device we can increase our reaction time and driving accuracy.\n\nFinally, we do not need to create a creature like ourselves. In fact, we can make the same being (human) mentally stronger.\n\nDo you agree with my view?",futurology
j9t4c5,"I mean I don’t want to sound like an edgy teen asshole from r/nihilism but, is humanity going to go extinct soon? I heard somewhere that sentient life has a shorter lifespan if you will than non sentient species. Idk that much about the subject and would like to know what you guys have to say.",futurology
j9racu,"I don't have the ability to show this using physics equations so please forgive me if I misphrase things. I was thinking about this amazing theory Penrose had, and it got me to wondering if for example a civilization that had become say a type 4 civilization might not be able to fully master Black Holes. As in feed one matter or e

In [95]:
# Check out the non-null body text

for i in range(5):
    display(non_null_body.iloc[i, :])

printmd('<br>')

print(non_null_body[-5:])


body         The more I look into AI and new projects like GPT-3, we humans suck. Sure GPT-3 isn't on the level as a human brain, but that doesn't mean gpt-4 or gpt-5 won't be.  I can already see the massive jobs loss as AI will offset more jobs than it will create. I'm just feeling a little existential crisis here
subreddit                                                                                                                                                                                                                                                                                                         futurology
Name: japxs3, dtype: object

body         As our dependence on computer technologies increases, will kids who were not allowed to use electronics (phones, computers, tablets) as toddlers become stunted because they wouldn't know how to use computers?\n\nWill computers be so ingrained in society that toddlers need to learn how to use them just as how they need to learn language?\n\nBy this metric, is it child abuse if you DON'T teach your kids how to use computers?
subreddit                                                                                                                                                                                                                                                                                                                                                                                                                                    futurology
Name: jamgnw, dtype: object

body         Please post all climate change news here unless the submission is an unique event that is a global headline across several trusted news sources.
subreddit                                                                                                                                          futurology
Name: jag7ht, dtype: object

body         Is there any effort by others to bring the other technologies such as, Molecular Manufacturing into reality?
subreddit                                                                                                      futurology
Name: ja7ygc, dtype: object

body         Everyday we're learing more efficient ways to synthesize food. Eventually there's going to be a synthetic dietary movement similar to vegetarianism and veganism. Probably sooner rather than later. And it won't be restricted to meat, synthetic vegetables and fruit will be included.\n\nSo what would a term for people who only eat synthetic food be? Artificivore? Synthetivore? Synthetarian? \n\nPitch ideas bellow.
subreddit                                                                                                                                                                                                                                                                                                                                                                                                                        futurology
Name: ja5exx, dtype: object

<br>